<a href="https://colab.research.google.com/github/sadrireza/Neural-Networks/blob/main/OCCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,  BatchNormalization, LeakyReLU,Dropout,ReLU
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.metrics import accuracy_score

In [5]:
!ls /content

 ACCURACY_OCCNN.py	     OCCNN_3.py
 ANOMALY_Z.npy		     sample_data
 FEATURES_COMP.npy	    'To relate the provided code to the PDF document you.docx'
 FEATURE_TEST_COMP.npy	     Usermanual.docx
 FEATURE_TRAINING_COMP.npy


In [6]:
# Loading Data
training = np.load("FEATURE_TRAINING_COMP.npy")
test = np.load("FEATURE_TEST_COMP.npy")
X_Z = np.load("ANOMALY_Z.npy")

# New section

In [7]:
# Label Creation
X = np.append(training,X_Z,axis=0)
L_1 = [1,0]
L_2 = [0,1]
Label_1 = [L_1 for i in range(len(training))]
Label_2 = [L_2 for i in range(len(X_Z))]
Label_1 = np.array(Label_1)
Label_2 = np.array(Label_2)
Label = np.append(Label_1, Label_2, axis=0)
hidden_layer = 50

In [8]:
# Model Creation
tf.keras.backend.clear_session()
model = Sequential(
    [
        Dense(hidden_layer,input_shape=(3,), activation="relu", name="Input_Layer"),
        Dense(hidden_layer, activation="relu", name="layer2"),
        Dense(2,activation="softmax", name="Last_layer"),
    ]
)

opt = tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(
    loss='binary_crossentropy',
    metrics=['accuracy'],
    optimizer=opt,
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_Layer (Dense)         (None, 50)                200       
                                                                 
 layer2 (Dense)              (None, 50)                2550      
                                                                 
 Last_layer (Dense)          (None, 2)                 102       
                                                                 
Total params: 2852 (11.14 KB)
Trainable params: 2852 (11.14 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
#First Training
kf = KFold(n_splits=5,shuffle=True) #Cross Validation to avoid biases in the designed classifications
kf.split(X)
accuracy_model = []
z = 0
for train_index, test_index in kf.split(X):
    # Split train-test
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Label[train_index],Label[test_index]
    # Train the model
    print(X_train.shape)
    train_report = model.fit(X_train, y_train, epochs=500, validation_data=(X_test, y_test), verbose=0)
    # Append to accuracy_model the accuracy of the model
    print("******Evaluate on test data with the model:**********",z)
    z = z+1
    results = model.evaluate(X_test, y_test)
    print("test loss, test acc:", results)
    print(train_report.history.keys())

(6720, 3)
******Evaluate on test data with the model:********** 0
53/53 [==============================] - 0s 2ms/step - loss: 0.0757 - accuracy: 0.9744
test loss, test acc: [0.07574070990085602, 0.9744047522544861]
dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
(6720, 3)
******Evaluate on test data with the model:********** 1
53/53 [==============================] - 0s 2ms/step - loss: 0.0710 - accuracy: 0.9768
test loss, test acc: [0.07096181809902191, 0.9767857193946838]
dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
(6720, 3)
******Evaluate on test data with the model:********** 2
53/53 [==============================] - 0s 2ms/step - loss: 0.0681 - accuracy: 0.9774
test loss, test acc: [0.06813709437847137, 0.9773809313774109]
dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
(6720, 3)
******Evaluate on test data with the model:********** 3
53/53 [==============================] - 0s 3ms/step - loss: 0.0616 - accuracy: 0.9821
test loss, test acc

In [10]:
# Label Test Creation

n_good = 4922-3400
n_bad = len(test)-n_good
Lable_1 = [1,0]
Lable_2 = [0,1]
Label_1 = [Lable_1 for i in range(n_good)]

Label_2 = [Lable_2 for i in range(n_bad)]
Label_1 = np.array(Label_1)
Label_2 = np.array(Label_2)
Label = np.append(Label_1, Label_2, axis=0)
model.evaluate(test, Label)

71/71 [==============================] - 0s 3ms/step - loss: 0.1053 - accuracy: 0.9551


[0.10525011271238327, 0.9551310539245605]

In [11]:
#Cycle 2
P_pred = model.predict(X_Z)
Adversal = X_Z[P_pred[:,0]<0.5,:]
len(Adversal)

#fig = plt.figure(figsize=(10, 10))
#ax = fig.add_subplot(111, projection='3d')
#ax.scatter(Adversal[:,0], Adversal[:,1], Adversal[:,2], alpha=0.5, color='red')
#ax.set_xlabel('f1')
#ax.set_ylabel('f2')
#ax.set_zlabel('TDT2')
#ax.set_xlim(-1, 1)
#ax.set_ylim(-1, 1)
#ax.set_zlim(-1, 1)
#ax.set_title('Feature space')
#plt.show()


157/157 [==============================] - 0s 1ms/step


4855

In [ ]:
#Preparation of Training Cycle 2
X = np.append(training,Adversal,axis=0)
L_1 = [1, 0]
L_2 = [0, 1]
Label_1 = [L_1 for i in range(len(training))]
Label_2 = [L_2 for i in range(len(Adversal))]
Label_1 = np.array(Label_1)
Label_2 = np.array(Label_2)
Label = np.append(Label_1, Label_2, axis=0)

X_train, X_test, y_train, y_test = train_test_split(X, Label, test_size=0.3)

kf = KFold(n_splits=5,shuffle=True)
kf.split(X)
accuracy_model = []
z = 0
for train_index, test_index in kf.split(X):
    # Split train-test
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Label[train_index],Label[test_index]
    # Train the model
    print(X_train.shape)
    train_report = model.fit(X_train, y_train, epochs=500, validation_data=(X_test, y_test), verbose=0)
    # Append to accuracy_model the accuracy of the model
    print("******Evaluate on test data with the model:**********",z)
    z = z+1
    results = model.evaluate(X_test, y_test)
    print("test loss, test acc:", results)
    print(train_report.history.keys())

model.save('OCC_50_50_2C')

(6604, 3)
******Evaluate on test data with the model:********** 0
52/52 [==============================] - 0s 2ms/step - loss: 0.0140 - accuracy: 0.9933
test loss, test acc: [0.014032633975148201, 0.9933373928070068]
dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
(6604, 3)
******Evaluate on test data with the model:********** 1
52/52 [==============================] - 0s 2ms/step - loss: 0.0079 - accuracy: 0.9970
test loss, test acc: [0.007874536328017712, 0.9969715476036072]
dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
(6604, 3)
******Evaluate on test data with the model:********** 2
52/52 [==============================] - 0s 2ms/step - loss: 0.0069 - accuracy: 0.9976
test loss, test acc: [0.006869628559798002, 0.9975772500038147]
dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
(6604, 3)
